In [218]:
import ffmpeg
import gradio as gr

from pathlib import Path
from datetime import datetime
from gradio_videogallery import videogallery
from projetMAB1.model import Model
from projetMAB1.indexer import index
from projetMAB1.assembler import assembly

In [219]:
VIDEO_FOLDER_PATH = "../../data/runtime/videos/"
EXPORT_FOLDER_PATH = "../../data/runtime/export/"
model = Model()

In [220]:
def query(text):
    matches = model.query(text)
    path_video_matches = [video_path for video_path, _ in matches]
    video_matches = [[Path(video)] for video in path_video_matches]
    resume_video = assembly(path_video_matches)
    print(video_matches)
    return gr.Dataset(samples=video_matches), resume_video

In [221]:
videos = sorted(Path("../../data/runtime/videos/").glob("*.mp4"))
images = sorted(Path("../../data/runtime/frames/").glob("*.png"))
fragments = sorted(Path("../../data/runtime/fragments").glob("*.mp4"))
images_by_video = {}
fragments_by_video = {}
video_by_images = {}

for image in images:

    key = image.stem.split("_")[0]
    if key not in images_by_video:
        images_by_video[key] = [image]
    else:
        images_by_video[key].append(image)
    video_by_images[image] = key

for fragment in fragments:

    key = fragment.stem.split("_")[0]
    if key not in fragments_by_video:
        fragments_by_video[key] = [fragment]
    else:
        fragments_by_video[key].append(fragment)

In [222]:
def export(gallery, evt: gr.SelectData):
    video_name, start_frame, end_frame = Path(gallery[evt.index]["name"]).stem.split("_")
    in_file = ffmpeg.input(f"{VIDEO_FOLDER_PATH}{video_name}.mp4", loglevel='warning', copyts=None)
    out_file = f"{EXPORT_FOLDER_PATH}{datetime.now()}.mp4"
    (
        ffmpeg.concat(in_file.trim(start_frame=start_frame, end_frame=end_frame),
                      )
        .filter('stepts', expr='PTS-STARTPTS')
        .ouput(out_file).run()
    )

In [223]:
examples = [
    ["A tennis match"],
	["A footbal match"],
	["A baseball match"],
	["Cooking"],
	["A video game"],
	["A political man"],
	["A political debate"],
	["Barack Obama"],
	["Abraham Lincoln"],
	["A concert"],
	["A computer screen"],
	["Advertising"],
	["Dragon ball"],
	["A pair of shoes"],
	["Elephants"],
	["An iPhone"],
	["A video clip"],
	["A drawing"],
	["A film"],
	["A makeup tutorial"],
	["Industrial machinery"],
	["A person repairing a car"]
]

In [224]:
css = "footer {display:none !important;} .gradio-container {min-height: 0px !important;}"
with gr.Blocks(theme=gr.themes.Base()) as demo:

    gr.Markdown("# Montage vidéo")

    with gr.Tab("Indexation"):
        video_to_index = gr.Video()
        progress = gr.Progress()
        index_button = gr.Button("Démarrer l'indexation de la vidéo")

    with gr.Tab("Gallerie"):
        videos = sorted(Path("../../data/runtime/videos/").glob("*.mp4"))
        for video in videos:
            with gr.Row():
                with gr.Column():
                    gr.Video(str(video), label=video.stem)
                with gr.Column():
                    print([[x] for x in fragments_by_video[video.stem]])
                    ds = gr.Dataset(
                        components=[gr.Video(visible=False)],
                        label=video.name,
                        samples=[[x] for x in fragments_by_video[video.stem]]
                    )

    with gr.Tab("Résumé vidéo"):
        resume_text = gr.TextArea(placeholder="Ecrivez ici le résumé en anglais", label="Résumé")
        query_button = gr.Button("Construire la vidéo")
        matches = gr.Dataset(label="Les morceaux de vidéos correspondant à la requête", components=[gr.Video(visible=False)], samples=[['../../data/runtime/videos/unicorn.mp4']])
        video_resume = gr.Video(label="Vidéo résumé")

    index_button.click(index, inputs=video_to_index)
    query_button.click(query, inputs=resume_text, outputs=[matches, video_resume])
    resume_text.submit(query, inputs=resume_text, outputs=[matches, video_resume])

demo.launch()

[[PosixPath('../../data/runtime/fragments/unicorn_0_333.mp4')]]
[[PosixPath('../../data/runtime/fragments/video0_0_52.mp4')], [PosixPath('../../data/runtime/fragments/video0_175_286.mp4')], [PosixPath('../../data/runtime/fragments/video0_286_301.mp4')], [PosixPath('../../data/runtime/fragments/video0_52_175.mp4')]]
[[PosixPath('../../data/runtime/fragments/video1_0_106.mp4')], [PosixPath('../../data/runtime/fragments/video1_106_194.mp4')], [PosixPath('../../data/runtime/fragments/video1_194_473.mp4')]]
[[PosixPath('../../data/runtime/fragments/video2_0_212.mp4')], [PosixPath('../../data/runtime/fragments/video2_212_341.mp4')], [PosixPath('../../data/runtime/fragments/video2_341_423.mp4')], [PosixPath('../../data/runtime/fragments/video2_423_492.mp4')], [PosixPath('../../data/runtime/fragments/video2_492_579.mp4')], [PosixPath('../../data/runtime/fragments/video2_579_623.mp4')]]
[[PosixPath('../../data/runtime/fragments/video3_0_98.mp4')], [PosixPath('../../data/runtime/fragments/video3

[[PosixPath('../../data/runtime/fragments/video0_286_301.mp4')], [PosixPath('../../data/runtime/fragments/video0_0_52.mp4')], [PosixPath('../../data/runtime/fragments/video0_175_286.mp4')]]
